In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))
import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
import os
import sys
sys.path.append('./')
sys.path.append('./model')
sys.path.append('./cost')
sys.path.append('./constraints')
sys.path.append('./utils')

In [ ]:
import UnicycleModel
import UnicycleCost
import UnicycleConstraints
from scipy.integrate import solve_ivp
from PTR import PTR
from LMI import Q_update
from Scaling import TrajectoryScaling
from matplotlib.patches import Ellipse
from utils_plot import plot_traj,plot_traj_set,plot_state_input
from Scaling import TrajectoryScaling
from utils_alg import get_neighbor_vec
import cvxpy as cvx
from utils_alg import get_sample_eta_w
from MSampling import Mstep

In [ ]:
ix = 3
iu = 2
iw = 2
iq = 2
ip = 2
N = 30
tf = 3
delT = tf/N
max_iter_SCP = 50

In [ ]:
# structued nonlinearity
C_o = np.array([[0,0,1],[0,0,0]])
D_o = np.array([[0,0],[1,0]])
E_o = np.array([[1,0],[0,1],[0,0]])
F = np.array([[0.1,0],[0,0.1],[0,0]])
G_o = np.zeros((iq,iw))
C = np.tile(C_o,(ip,1)) 
D = np.tile(D_o,(ip,1))
G = np.tile(G_o,(ip,1)) 
E = np.repeat(E_o,iq,axis=1)

## obstacle setting

In [ ]:
def get_H_obs(rx,ry) :
    return np.diag([1/rx,1/ry])
# obstacle
c_list = []
H_list = []
c1 = [1,2]
H1 = get_H_obs(0.75,1.5)
c_list.append(c1)
H_list.append(H1)
c2 = [4,3]
H2 = get_H_obs(0.75,1.5)
c_list.append(c2)
H_list.append(H2)

In [ ]:
# plt.figure(figsize=(5,8))
# ax=plt.gca()
# for ce,H in zip(c_list,H_list) :
#     rx = 1/H[0,0]
#     ry = 1/H[1,1]
#     circle1 = Ellipse((ce[0],ce[1]),rx*2,ry*2,color='tab:red',alpha=0.5,fill=True)
#     ax.add_patch(circle1)
# plt.axis([-0.5, 5.5, -0.5, 5.5])
# plt.gca().set_aspect('equal', adjustable='box')
# plt.grid(True)

## initial and final conditions 

In [ ]:

xi = np.zeros(3)
xi[0] = 0.0
xi[1] = 0.0 
xi[2] = 0

xf = np.zeros(3)
xf[0] = 5.0
xf[1] = 5.0
xf[2] = 0

xi_margin = np.array([0.1,0.1,np.deg2rad(5)])*2
Qf = np.diag([0.5**2,0.5**2,np.deg2rad(20)**2])*1.5
vec_neighbor = []
xi_neighbor = np.array(get_neighbor_vec(-1,0,ix,None,vec_neighbor)) * np.array(xi_margin) + xi

myModel = UnicycleModel.unicycle('Hello',ix,iu,'numeric_central')
myCost = UnicycleCost.unicycle('Hello',ix,iu,N)
myConst = UnicycleConstraints.UnicycleConstraints('Hello',ix,iu)
myConst.set_obstacle(c_list,H_list)

x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf
# u0 = np.random.rand(N,iu)
u0 = np.zeros((N+1,iu))



# Initial setting

## initial trajectory x,u

In [ ]:
i1 = PTR('unicycle',N,tf,max_iter_SCP,myModel,myCost,myConst,type_discretization="zoh",
          w_c=1,w_vc=1e3,w_tr=1e-1,tol_vc=1e-6,tol_tr=1e-5,verbosity=False)
x,u,xbar,ubar,total_num_iter,flag_boundary,l,l_vc,l_tr = i1.run(x0,u0,xi,xf)
# plot_traj(xbar,ubar,c_list,H_list,xf)
# plot_state_input(xbar,ubar,xi,xf,N,delT)

## Initial trajectory Q, K

In [ ]:
# get LQR K
# A,B,s,z,vc = i1.get_model()
# C = s+z
A,B = myModel.diff_numeric_central(xbar,ubar)
from utils_alg import get_K
S = 2*np.eye(ix)
R = np.eye(iu)
# R[1,1] *= 0.1
S_final = 2*np.eye(ix)
Kbar = get_K(A[:N],B[:N],S,R,N,ix,iu)
Qbar = np.tile(np.diag([0.5**2,0.5**2,np.deg2rad(20)**2]),(N+1,1,1))

In [ ]:
plot_traj_set(xbar,ubar,c_list,H_list,Qbar,xi=xi,xf=xf,xi_margin=xi_margin,Qf=Qf,idx_plot=0)


# Start

In [ ]:
history = []
max_iter = 50
for _ in range(max_iter) :
    traj = {}
    # step 1
    x0 = xbar
    u0 = ubar

    # SCP
    i1 = PTR('unicycle',N,tf,max_iter_SCP,myModel,myCost,myConst,type_discretization="zoh",
              w_c=1,w_vc=1e3,w_tr=1e1,tol_vc=1e-6,tol_tr=1e-5,verbosity=False)
    _,_,x,u,total_num_iter,flag_boundary,l,l_vc,l_tr = i1.run(x0,u0,xi,xf,Qbar,Kbar)#,xi_neighbor,xf_margin)
    # plot_traj_set(x,u,c_list,H_list,Qbar,xi=xi,xf=xf,xi_margin=xi_margin,Qf=Qf,idx_plot=0)

    # step 2
    A,B = myModel.diff_numeric_central(xbar,ubar)
    myM = Mstep(ix,iu,iq,ip,iw,N)
    myM.initialize(xbar,ubar,Qbar,Kbar,A,B,C,D,E,F,G,myModel)
    gamma = myM.update_parallel()
#     gamma = np.ones((N,ip*iq))*np.array([0.39668553,0.12631765,0.42389774,0.13292677])*2
    print("mean of gamma",np.mean(gamma,0))
    print("max of gamma",np.max(gamma,0))
    # step 3
    myQ = Q_update(ix,iu,iq,ip,iw,N,delT,myCost.S,myCost.R)
    myQ.initialize(x,u,A,B,C,D,E,F,G)

    Q,K,Y,status = myQ.solve_type_1(gamma,xi_neighbor,Qf,Qbar)
    print("LMI status:" + status)

    # plot the trajectory
    plot_traj_set(x,u,c_list,H_list,Q,xi=xi,xf=xf,xi_margin=xi_margin,Qf=Qf,idx_plot=0)
    plt.show()
    # show the difference
    x_diff = np.mean(np.linalg.norm(x-xbar,2,1))
    u_diff = np.mean(np.linalg.norm(u[:N]-ubar[:N],2,1))
    Q_diff = np.mean([np.linalg.norm(Q[i]-Qbar[i],'fro') for i in range(N+1)])
    K_diff = np.mean([np.linalg.norm(K[i]-Kbar[i],'fro') for i in range(N)])
    print("x {:8.6f} u {:8.6f} Q {:8.6f} K {:8.6f}".format(x_diff,u_diff,Q_diff,K_diff))

    # save trajectory
    traj['x'] = x
    traj['u'] = u
    traj['Q'] = Q
    traj['K'] = K
    traj['gamma'] = gamma
    history.append(traj)
    
    # update trajectory
    Qbar = Q
    Kbar = K
    xbar = x
    ubar = u

In [ ]:
# show the difference
x_distance = [np.mean(np.linalg.norm(history[i]['x']-history[-1]['x'],2,1)) / 
      np.mean(np.linalg.norm(history[-1]['x'],2,1))
      for i in range(max_iter-1)]
u_distance = [np.mean(np.linalg.norm(history[i]['u']-history[-1]['u'],2,1)) / 
      np.mean(np.linalg.norm(history[-1]['u'],2,1))
      for i in range(max_iter-1)]
# K_diff = np.mean([np.linalg.norm(K[i]-Kbar[i],'fro') for i in range(N)])

In [ ]:
Q_last_norm = np.mean([np.linalg.norm(history[-1]['Q'][i],'fro') for i in range(N)])
Q_distance = np.array([np.mean([np.linalg.norm(history[j]['Q'][i]-history[-1]['Q'][i],'fro') for i in range(N)]) 
          for j in range(max_iter-1)])/Q_last_norm
K_last_norm = np.mean([np.linalg.norm(history[-1]['K'][i],'fro') for i in range(N)])
K_distance = np.array([np.mean([np.linalg.norm(history[j]['K'][i]-history[-1]['K'][i],'fro') for i in range(N)]) 
          for j in range(max_iter-1)])/K_last_norm


In [ ]:
fS = 15
plt.figure(figsize=(15,10))
plt.subplot(221)
plt.plot([i+1 for i in range(max_iter-1)],x_distance,'o-')
plt.xlabel('iteration number',fontsize=fS)
plt.ylabel(r'$\frac{||x^i - x^*||_2}{||x^*||_2}$',fontsize=fS)
plt.yscale('log')
plt.subplot(222)
plt.plot([i+1 for i in range(max_iter-1)],u_distance,'o-')
plt.xlabel('iteration number',fontsize=fS)
plt.ylabel(r'$\frac{||u^i - u^*||_2}{||u^*||_2}$',fontsize=fS)
plt.yscale('log')
plt.subplot(223)
plt.plot([i+1 for i in range(max_iter-1)],Q_distance,'o-')
plt.xlabel('iteration number',fontsize=fS)
plt.ylabel(r'$\frac{||Q^i - Q^*||_F}{||Q^*||_F}$',fontsize=fS)
plt.yscale('log')
plt.subplot(224)
plt.plot([i+1 for i in range(max_iter-1)],K_distance,'o-')
plt.xlabel('iteration number',fontsize=fS)
plt.ylabel(r'$\frac{||K^i - K^*||_F}{||K^*||_F}$',fontsize=fS)
plt.yscale('log')

# is it really.. invariant?

In [ ]:
xbar = history[-1]['x']
ubar = history[-1]['u']
Qbar = history[-1]['Q']
Kbar = history[-1]['K']

In [ ]:
from utils_alg import forward_full_with_K,get_sample_trajectory
import scipy

x0_sample = []
# x0_sample.append(xbar[0])
num_sample = 100
idx = 0
for i in range(num_sample) :
    z = np.random.randn(ix)
    z = z / np.linalg.norm(z)
    x_s = xbar[idx] + scipy.linalg.sqrtm(Qbar[idx])@z
    x0_sample.append(x_s)
    



In [ ]:
# xsam,usam = forward_full_with_K(x0_sample[98],x,u,K,myModel,N,ix,iu,delT)
xsam,usam = get_sample_trajectory(xi,x0_sample,xbar,ubar,Kbar,myModel,N,ix,iu,iw,delT,flag_noise=True)

In [ ]:
plot_traj_set(xbar,ubar,c_list,H_list,Qbar,xi=xi,xf=xf,xi_margin=xi_margin,Qf=Qf,idx_plot=0)
for xsam_e in xsam :
# xsam_e = xsam[1]
    plt.plot(xsam_e[:,0], xsam_e[:,1],'-',markersize=4.0, linewidth=1.0,color='tab:purple')
plt.plot(1e3,1e3,'-',color='tab:purple',label='trajectory samples')
plt.legend(fontsize=15)

In [ ]:
xsam_e = xsam[0]
usam_e = usam[0]
plot_state_input(xsam_e,usam_e,None,None,N,delT)